# Library

In [1]:
import camelot
import pandas as pd
import os
import numpy as np
from itertools import product
import cv2
import pytesseract
from paddleocr import PaddleOCR
import re
# import matplotlib.pyplot as plt


pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

pd.options.display.float_format = '{:,.4f}'.format

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

d:\anaconda\envs\camelot_env\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
d:\anaconda\envs\camelot_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Function for PDF Reading

In [2]:
def extract_pdf(tables) :
    # Dictionary to store DataFrames for each page
    page_dfs = {}

    # Group tables by page
    for table in tables:
        page_number = table.page  # which page the table came from
        if page_number not in page_dfs:
            page_dfs[page_number] = []

        # Convert the table to a DataFrame and append
        df = table.df
        page_dfs[page_number].append(df)
    
    return(page_dfs)

In [3]:
def search_page(page_dfs, target_value, row_range, col_range):
    # df to store the found DataFrames with custom names
    found_dfs = {}
    found_keys = []
    found_page_num = []

    # Loop through all pages and tables
    for page_num, dfs in page_dfs.items():
        for i, df in enumerate(dfs, start=1):
            # Slice rows
            start_row, end_row = row_range
            sub_df = df.iloc[start_row:end_row + 1]

            # Slice columns
            start_col, end_col = col_range
            sub_df_cols = sub_df.iloc[:, start_col:end_col + 1]

            # Check if target_value exists exactly in any cell
            match_found = (sub_df_cols == target_value).any().any()

            if match_found:
                custom_name = f"{target_value}_page{page_num}_table{i}"
                found_keys.append(custom_name)
                found_page_num.append(page_num)
                found_dfs[custom_name] = df.copy()
                print(f"Found exact match '{target_value}' in Page {page_num}, Table {i}. Saved as '{custom_name}'.")
    
    return found_dfs, found_keys, found_page_num

In [4]:
def normalize_text(text):
    """Convert text to lowercase and remove all spaces."""
    return str(text).lower().replace(" ", "")

def merge_rows(df, merge_list, col_index=1):
    # Normalize the merge_list
    normalized_list = [normalize_text(item) for item in merge_list]

    rows_to_drop = []
    for i in range(len(df) - 1):
        current_val = normalize_text(df.iloc[i, col_index])

        if current_val in normalized_list:
            # Merge current row with the next row (column by column)
            df.iloc[i] = df.iloc[i].astype(str) + " " + df.iloc[i + 1].astype(str)
            rows_to_drop.append(i + 1)

    df = df.drop(rows_to_drop).reset_index(drop=True)
    return df

def merge_2_rows(df, two_row_list, col_index=1):
    """
    Merge rows where the value in `col_index` is in `two_row_list`
    with the next 2 rows, and then remove those 2 rows.
    """
    # Normalize the list for comparison
    normalized_list = [normalize_text(item) for item in two_row_list]
    rows_to_drop = []

    for i in range(len(df)):
        if i in rows_to_drop:
            continue  # skip already marked rows

        current_val = normalize_text(df.iloc[i, col_index])
        if current_val in normalized_list and i + 2 < len(df):
            for col in df.columns:
                vals = [
                    str(df.iloc[i, col]).strip(),
                    str(df.iloc[i + 1, col]).strip(),
                    str(df.iloc[i + 2, col]).strip()
                ]
                df.iat[i, col] = " ".join([v for v in vals if v])

            print(f"Merging rows {i + 1} and {i + 2} into row {i + 1} for value '{df.iloc[i, col_index]}'")
            rows_to_drop.extend([i + 1, i + 2])

    print(f"Dropping rows: {rows_to_drop}")
    return df.drop(rows_to_drop).reset_index(drop=True)

def merge_3_rows(df, three_row_list, col_index=1):
    """
    Merge rows where the value in `col_index` is in `three_row_list`
    with the next 3 rows, and then remove those 3 rows.
    """
    # Normalize the list for comparison
    normalized_list = [normalize_text(item) for item in three_row_list]
    rows_to_drop = []

    for i in range(len(df)):
        if i in rows_to_drop:
            continue  # skip already marked rows

        current_val = normalize_text(df.iloc[i, col_index])
        if current_val in normalized_list and i + 3 < len(df):
            for col in df.columns:
                vals = [
                    str(df.iloc[i, col]).strip(),
                    str(df.iloc[i + 1, col]).strip(),
                    str(df.iloc[i + 2, col]).strip(),
                    str(df.iloc[i + 3, col]).strip()
                ]
                df.iat[i, col] = " ".join([v for v in vals if v])

            print(f"Merging rows {i + 1}, {i + 2}, {i + 3} into row {i + 1} for value '{df.iloc[i, col_index]}'")
            rows_to_drop.extend([i + 1, i + 2, i + 3])

    print(f"Dropping rows: {rows_to_drop}")
    return df.drop(rows_to_drop).reset_index(drop=True)


In [5]:
def tune_camelot_params(pdf_path, target_value, pages="all"):
    """
    Tune Camelot parameters to find the correct table from a PDF.

    Parameters
    ----------
    pdf_path : str
        Path to the PDF file.
    searching_function : callable
        A function that takes Camelot tables and returns True if the target is found.
    pages : str
        Which pages to parse. Default is 'all'.

    Returns
    -------
    tables : camelot.core.TableList or None
        The extracted tables if found.
    best_params : dict or None
        The parameters that successfully extracted the desired content.
    """

    row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
    col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

    edge_tol_values = [50, 100, 150, 200, 300]
    row_tol_values = [1, 2, 3, 4, 5]
    split_text_values = [True, False]
    strip_text_values = ['\n', '']

    results = []

    for edge_tol, row_tol, split_text, strip_text in product(
        edge_tol_values, row_tol_values, split_text_values, strip_text_values
    ):
        try:
            print(f"Trying: edge_tol={edge_tol}, row_tol={row_tol}, split_text={split_text}, strip_text={repr(strip_text)}")
            tables = camelot.read_pdf(
                pdf_path,
                flavor="stream",
                pages=pages,
                edge_tol=edge_tol,
                row_tol=row_tol,
                split_text=split_text,
                strip_text=strip_text,
            )

            page_dfs = extract_pdf(tables)
            found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
            
            if len(found_keys)>0:
                print(f"✅ Match found for parameters: edge_tol={edge_tol}, row_tol={row_tol}")
                results.append((
                    tables,
                    {
                        "edge_tol": edge_tol,
                        "row_tol": row_tol,
                        "split_text": split_text,
                        "strip_text": strip_text,
                    }
                ))
        except Exception as e:
            print(f"❌ Error with edge_tol={edge_tol}, row_tol={row_tol}: {e}")
            continue

    if not results:
        print("❌ No matching table found.")
    else:
        print(f"Found {len(results)} matching parameter combinations.")

    return results

# Read PDF

## Rasio

### BCA Digital

In [ ]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\BCA Digital'

# PDF list to be processed
pdf_list = [
    "BCADigitalq42024.pdf",
    "BCADigitalq32024.pdf",
    "BCADigitalq22024.pdf",
    "BCADigitalq12024.pdf",
    "BCADigitalq42023.pdf",
    "BCADigitalq32023.pdf",
    "BCADigitalq22023.pdf",
    "BCADigitalq12023.pdf"
]

# Your search parameters
target_value = "RASIO"       # The value to search for
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

rasio_1_row_col_1_merge = [
    "RASIO",
    "Kewajiban Penyediaan Modal",
    "Aset produktif bermasalah terhadap",
    "Biaya Operasional terhadap",
    "Posisi Devisa Neto (PDN) secara"
]
rasio_2_row_col_1_merge = [
    "Aset produktif bermasalah dan aset",
    "Cadangan Kerugian Penurunan Nilai",
]

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=200,
        row_tol=5, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_rasio = found_dfs[found_keys[0]] if found_keys else None

    # Apply the function
    df_merged_1 = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)
    df_merged_2 = merge_rows(df_merged_1.copy(), rasio_1_row_col_1_merge, col_index=1)
    df_result = merge_2_rows(df_merged_2.copy(), rasio_2_row_col_1_merge, col_index=1)

    result_path = os.path.join(path, f"rasio_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

### Seabank

In [ ]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Seabank'

# PDF list to be processed
pdf_list = [
    "Seabankq42024.pdf",
    "Seabankq32024.pdf",
    "Seabankq22024.pdf",
    "Seabankq12024.pdf",
    "Seabankq42023.pdf",
    "Seabankq32023.pdf",
    "Seabankq22023.pdf",
    "Seabankq12023.pdf"
]

rasio_1_row_col_1_merge = [
    "KewajibanPenyediaanModal",
    "Asetproduktifbermasalah",
    "BebanOperasionalterhadap",
    "PosisiDevisaNeto(PDN)secara"
]
rasio_2_row_col_1_merge = [
    "CadanganKerugianPenurunan"
]
rasio_3_row_col_1_merge = [
    "Asetproduktifbermasalahdan"
]

# Your search parameters
target_value = "Rasio"       # The value to search for
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=200,
        row_tol=8, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_rasio = found_dfs[found_keys[0]] if found_keys else None

    # Apply the function
    df_merged_1 = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)
    df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    df_result = merge_3_rows(df_merged_2.copy(), rasio_3_row_col_1_merge, col_index=1)

    result_path = os.path.join(path, f"rasio_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


Found exact match 'Rasio' in Page 8, Table 1. Saved as 'Rasio_page8_table1'.
Merging rows 12 and 13 into row 12 for value 'CadanganKerugianPenurunan Nilai(CKPN)asetkeuangan terhadap aset produktif *'
Dropping rows: [12, 13]
Merging rows 7, 8, 9 into row 7 for value 'Asetproduktifbermasalahdan asetnon-produktifbermasalah terhadaptotalasetproduktifdan aset non-produktif'
Dropping rows: [7, 8, 9]


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value
d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (36.919999999999995, 214.02398899999994, 578.9808444677496, 416.2054914695122)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)


Found exact match 'Rasio' in Page 8, Table 1. Saved as 'Rasio_page8_table1'.
Merging rows 12 and 13 into row 12 for value 'CadanganKerugianPenurunan Nilai(CKPN)asetkeuangan terhadap aset produktif *'
Dropping rows: [12, 13]
Merging rows 7, 8, 9 into row 7 for value 'Asetproduktifbermasalahdan asetnon-produktifbermasalah terhadaptotalasetproduktifdan aset non-produktif'
Dropping rows: [7, 8, 9]


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


...

Found exact match 'Rasio' in Page 8, Table 1. Saved as 'Rasio_page8_table1'.
Merging rows 13 and 14 into row 13 for value 'CadanganKerugianPenurunan Nilai(CKPN)asetkeuangan terhadap aset produktif'
Dropping rows: [13, 14]
Merging rows 8, 9, 10 into row 8 for value 'Asetproduktifbermasalahdan asetnon-produktifbermasalah terhadaptotalasetproduktif dan aset non-produktif'
Dropping rows: [8, 9, 10]


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


Found exact match 'Rasio' in Page 8, Table 1. Saved as 'Rasio_page8_table1'.
Merging rows 10 and 11 into row 10 for value 'CadanganKerugianPenurunan Nilai(CKPN)asetkeuangan terhadap aset produktif'
Dropping rows: [10, 11]
Merging rows 5, 6, 7 into row 5 for value 'Asetproduktifbermasalahdan asetnon-produktifbermasalah terhadap totalasetproduktif dan aset non-produktif'
Dropping rows: [5, 6, 7]


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


Found exact match 'Rasio' in Page 8, Table 1. Saved as 'Rasio_page8_table1'.
Merging rows 10 and 11 into row 10 for value 'CadanganKerugianPenurunan Nilai(CKPN)asetkeuangan terhadap aset produktif'
Dropping rows: [10, 11]
Merging rows 5, 6, 7 into row 5 for value 'Asetproduktifbermasalahdan asetnon-produktifbermasalah terhadap totalasetproduktif dan aset non-produktif'
Dropping rows: [5, 6, 7]


### Allo Bank

In [ ]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Allobank'

# PDF list to be processed
pdf_list = [
    "Allobankq42024.pdf",
    "Allobankq32024.pdf",
    "Allobankq22024.pdf",
    "Allobankq12024.pdf",
    "Allobankq42023.pdf",
    "Allobankq32023.pdf",
    "Allobankq22023.pdf",
    "Allobankq12023.pdf"
]

rasio_1_row_col_1_merge = [
    "Rasio",
    "Aset produktif bermasalah dan aset non produktif bermasalah",
    "Cadangan Kerugian Penurunan Nilai (CKPN) aset keuangan",
]

# Your search parameters
target_value = "Rasio"       # The value to search for
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=200,
        row_tol=8, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_rasio = found_dfs[found_keys[0]] if found_keys else None

    # Apply the function
    # df_merged_1 = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    df_result = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)

    result_path = os.path.join(path, f"rasio_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'Rasio' in Page 10, Table 1. Saved as 'Rasio_page10_table1'.


d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'Rasio' in Page 10, Table 1. Saved as 'Rasio_page10_table1'.


d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'Rasio' in Page 10, Table 1. Saved as 'Rasio_page10_table1'.
Found exact match 'Rasio' in Page 10, Table 1. Saved as 'Rasio_page10_table1'.


d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'Rasio' in Page 10, Table 1. Saved as 'Rasio_page10_table1'.
Found exact match 'Rasio' in Page 10, Table 1. Saved as 'Rasio_page10_table1'.
Found exact match 'Rasio' in Page 10, Table 1. Saved as 'Rasio_page10_table1'.
Found exact match 'Rasio' in Page 10, Table 1. Saved as 'Rasio_page10_table1'.


### Bank Jago

In [ ]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Bank Jago'

# PDF list to be processed
pdf_list = [
    "BankJagoq42024.pdf",
    "BankJagoq12024.pdf",
    "BankJagoq42023.pdf",
    "BankJagoq22023.pdf"
]

rasio_1_row_col_1_merge = [
    "Aset produktif bermasalah dan aset non - produktif bermasalah  terhadap  total  aset  produktif  dan  aset",
    "Aset produktif bermasalah terhadap total aset",
    "Cadangan  Kerugian  Penurunan  Nilai (CKPN)  aset",
]

# Your search parameters
# target_value = "RASIO (%)"       # The value to search for
target_value = "Rasio Kinerja (Bank)"
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=50,
        row_tol=5, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_rasio = found_dfs[found_keys[0]] if found_keys else None

    # Apply the function
    # df_merged_1 = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    df_result = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)

    result_path = os.path.join(path, f"rasio_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Found exact match 'Rasio Kinerja (Bank)' in Page 1, Table 2. Saved as 'Rasio Kinerja (Bank)_page1_table2'.


In [94]:
def extract_numeric_percent_texts(image_path, ocr):
    """
    Extract PaddleOCR rec_texts containing both numbers and '%' character.
    Arrange them into a 2-column DataFrame (even index → Column 1, odd index → Column 2).
    """
    
    results = ocr.predict(image_path)

    # Collect all rec_texts from the results
    all_texts = []
    for res in results:
        rec_texts = res.get('rec_texts', [])
        all_texts.extend(rec_texts)

    # Filter texts that must contain at least one digit and a '%' character
    filtered_texts = [
        text for text in all_texts if re.search(r'\d', text) and '%' in text
    ]

    # Split into 2 columns (even index → col1, odd index → col2)
    col1 = filtered_texts[0::2]
    col2 = filtered_texts[1::2]

    # Pad columns if their lengths differ
    max_len = max(len(col1), len(col2))
    col1 += [''] * (max_len - len(col1))
    col2 += [''] * (max_len - len(col2))

    df = pd.DataFrame({'Column 1': col1, 'Column 2': col2})
    return df



In [95]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Bank Jago'
rasio_img_list = [
    "rasio_q1_2023.png",
    "rasio_q3_2023.png",
    "rasio_q2_2024.png",
    "rasio_q3_2024.png"
]

ocr = PaddleOCR(lang='en')
# List using OCR

for img_name in rasio_img_list:
    img_path = os.path.join(path, img_name)
    df_test = extract_numeric_percent_texts(img_path, ocr)
    result_path = os.path.join(path, f"rasio_ocr_merged_{img_name.replace('.png', '.xlsx')}")
    df_test.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in C:\Users\Sharing Vision\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 1429.55it/s]
Creating model: ('UVDoc', None)
The model(UVDoc) is not supported to run in MKLDNN mode! Using `paddle` instead!
Using official model (UVDoc), the model files will be automatically downloaded and saved in C:\Users\Sharing Vision\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 8123.25it/s]
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Using official model (PP-LCNet_x1_0_textline_ori), the model files will be automatically downloaded and saved in C:\Users\Sharing Vision\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 302.47it/s]
Creating model: ('PP-OCRv5_server_det', None)
Using official model (PP-OCRv5_server_det), the model files will be automaticall

## Aset

### BCA Digital

In [33]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\BCA Digital'

# PDF list to be processed
pdf_list = [
    "BCADigitalq42024.pdf",
    "BCADigitalq32024.pdf",
    "BCADigitalq22024.pdf",
    "BCADigitalq12024.pdf",
    "BCADigitalq42023.pdf",
    "BCADigitalq32023.pdf",
    "BCADigitalq22023.pdf",
    "BCADigitalq12023.pdf"
]

# Your search parameters
target_value = "ASET"       # The value to search for
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

rasio_1_row_col_1_merge = [
    "POS - POS",
]
# rasio_2_row_col_1_merge = [
#     "Aset produktif bermasalah dan aset",
#     "Cadangan Kerugian Penurunan Nilai",
# ]

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=200,
        row_tol=8, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_aset = found_dfs[found_keys[0]] if found_keys else None

    # Apply the function
    # df_merged_1 = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_rows(df_merged_1.copy(), rasio_1_row_col_1_merge, col_index=1)
    df_result = merge_rows(df_aset.copy(), rasio_1_row_col_1_merge, col_index=1)

    result_path = os.path.join(path, f"aset_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.


Cannot set gray non-stroke color because /'P28' is an invalid float value
Cannot set gray non-stroke color because /'P30' is an invalid float value


Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.


Cannot set gray non-stroke color because /'P28' is an invalid float value
Cannot set gray non-stroke color because /'P30' is an invalid float value


Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.


Cannot set gray non-stroke color because /'P28' is an invalid float value
Cannot set gray non-stroke color because /'P30' is an invalid float value


Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.


### Seabank

In [57]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Seabank'

# PDF list to be processed
pdf_list = [
    "Seabankq42024.pdf",
    "Seabankq32024.pdf",
    "Seabankq22024.pdf",
    "Seabankq12024.pdf",
    "Seabankq42023.pdf",
    "Seabankq32023.pdf",
    "Seabankq22023.pdf",
    "Seabankq12023.pdf"
]

rasio_1_row_col_1_merge = [
    "Tagihan atas surat berharga yang dibeli dengan janji dijual kembali (reverse ",
    "7. Tagihan atas surat berharga yang dibeli dengan janji dijual kembali (reverse"
]

# Your search parameters
target_value = "ASET"       # The value to search for
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=200,
        row_tol=7, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_aset = found_dfs[found_keys[0]] if found_keys else None

    # Apply the function
    df_merged_1 = merge_rows(df_aset.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    df_result = merge_rows(df_merged_1.copy(), rasio_1_row_col_1_merge, col_index=0)

    result_path = os.path.join(path, f"aset_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value
d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (36.919999999999995, 214.02398899999994, 578.9808444677496, 416.2054914695122)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)


Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


...

Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.


### Allo Bank

In [77]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Allobank'

# PDF list to be processed
pdf_list = [
    "Allobankq42024.pdf",
    "Allobankq32024.pdf",
    "Allobankq22024.pdf",
    "Allobankq12024.pdf",
    "Allobankq42023.pdf",
    "Allobankq32023.pdf",
    "Allobankq22023.pdf",
    "Allobankq12023.pdf"
]

rasio_1_row_col_1_merge = [
    "Tagihan atas surat berharga yang dibeli dengan janji dijual kembali",
]

# Your search parameters
target_value = "ASET"       # The value to search for
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=200,
        row_tol=8, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_aset = found_dfs[found_keys[0]] if found_keys else None

    # Apply the function
    df_merged_1 = merge_rows(df_aset.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    df_result = merge_rows(df_merged_1.copy(), rasio_1_row_col_1_merge, col_index=0)

    result_path = os.path.join(path, f"aset_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.


d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.


d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.
Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.


d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found exact match 'ASET' in Page 1, Table 2. Saved as 'ASET_page1_table2'.


### Bank Jago

In [ ]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Bank Jago'

# PDF list to be processed
pdf_list = [
    "BankJagoq42024.pdf",
    "BankJagoq12024.pdf",
    "BankJagoq42023.pdf",
    "BankJagoq22023.pdf",
    "BankJagoq12023.pdf"
]

rasio_1_row_col_1_merge = [
    "Surat berharga yang dijual dengan janji dibeli",
]

# Your search parameters
# target_value = "RASIO (%)"       # The value to search for
target_value = "ASET"
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=50,
        row_tol=5, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    print("Processing PDF:", pdf_name)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_aset = found_dfs[found_keys[0]] if found_keys else None
    if(len(found_keys) > 0) :
        print(f"Found {len(found_keys)} keys in {pdf_name}: {found_keys}")

    # Apply the function
    # df_merged_1 = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    df_result = merge_rows(df_aset.copy(), rasio_1_row_col_1_merge, col_index=1)

    result_path = os.path.join(path, f"aset_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Processing PDF: BankJagoq42024.pdf
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found 1 keys in BankJagoq42024.pdf: ['ASET_page1_table1']
Processing PDF: BankJagoq32024.pdf
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found 1 keys in BankJagoq32024.pdf: ['ASET_page1_table1']
Processing PDF: BankJagoq22024.pdf
Found exact match 'ASET' in Page 1, Table 4. Saved as 'ASET_page1_table4'.
Found 1 keys in BankJagoq22024.pdf: ['ASET_page1_table4']
Processing PDF: BankJagoq12024.pdf
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found 1 keys in BankJagoq12024.pdf: ['ASET_page1_table1']
Processing PDF: BankJagoq42023.pdf
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found 1 keys in BankJagoq42023.pdf: ['ASET_page1_table1']
Processing PDF: BankJagoq22023.pdf
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found 1 keys in BankJagoq22023.pdf: ['ASET_page1_t

In [20]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Bank Jago'

# PDF list to be processed
pdf_list = [
    "BankJagoq32023.pdf",
]

rasio_1_row_col_1_merge = [
    "Tagihan atas surat berharga yang dibeli dengan janji"
]

# Your search parameters
# target_value = "RASIO (%)"       # The value to search for
target_value = "ASET"
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=20,
        row_tol=5, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    print("Processing PDF:", pdf_name)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_aset = found_dfs[found_keys[0]] if found_keys else None
    if(len(found_keys) > 0) :
        print(f"Found {len(found_keys)} keys in {pdf_name}: {found_keys}")

    # Apply the function
    # df_merged_1 = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    df_result = merge_rows(df_aset.copy(), rasio_1_row_col_1_merge, col_index=1)

    result_path = os.path.join(path, f"aset_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Processing PDF: BankJagoq32023.pdf
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found 1 keys in BankJagoq32023.pdf: ['ASET_page1_table1']


In [27]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Bank Jago'

# PDF list to be processed
pdf_list = [
    "BankJagoq22024.pdf",
    "BankJagoq32024.pdf",
]

rasio_1_row_col_1_merge = [
    "7.  Tagihan atas surat berharga yang dibeli dengan janji",
    "13. Cadangan kerugian penurunan nilai aset"
]

# Your search parameters
# target_value = "RASIO (%)"       # The value to search for
target_value = "ASET"
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=20,
        row_tol=5, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    print("Processing PDF:", pdf_name)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_aset = found_dfs[found_keys[0]] if found_keys else None
    if(len(found_keys) > 0) :
        print(f"Found {len(found_keys)} keys in {pdf_name}: {found_keys}")

    # Apply the function
    # df_merged_1 = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    df_result = merge_rows(df_aset.copy(), rasio_1_row_col_1_merge, col_index=1)

    result_path = os.path.join(path, f"aset_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Processing PDF: BankJagoq32024.pdf
Found exact match 'ASET' in Page 1, Table 1. Saved as 'ASET_page1_table1'.
Found 1 keys in BankJagoq32024.pdf: ['ASET_page1_table1']


## Liabilitas

In [14]:
def cut_dataframe_target(df:pd.DataFrame, col_index:int, target:str)-> pd.DataFrame :
    standardized_target = target_value.lower().replace(' ', '')

    # Standardize column 1 values
    standardized_col = df[col_index].str.lower().str.replace(' ', '', regex=False)

    # Find first match
    start_index = df[standardized_col == standardized_target].index.min()

    # Filter rows from that index down
    if(start_index >= 3) :
        filtered_df = df.loc[start_index-3:]
    else :
        filtered_df = df.loc[0:]

    return(filtered_df)

### BCA Digital

In [38]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\BCA Digital'

# PDF list to be processed
pdf_list = [
    "BCADigitalq42024.pdf",
    "BCADigitalq32024.pdf",
    "BCADigitalq22024.pdf",
    "BCADigitalq12024.pdf",
    "BCADigitalq42023.pdf",
    "BCADigitalq32023.pdf",
    "BCADigitalq22023.pdf",
    "BCADigitalq12023.pdf"
]

# Your search parameters
target_value = "LIABILITAS"       # The value to search for
row_range = (0, 40)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

liabilitas_1_row_col_1_merge = [
    "POS - POS"
]

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=200,
        row_tol=5, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_liabilitas = found_dfs[found_keys[0]] if found_keys else None

    # Apply the function
    df_merged_1 = merge_rows(df_liabilitas.copy(), liabilitas_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_rows(df_merged_1.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_result = merge_2_rows(df_merged_2.copy(), rasio_2_row_col_1_merge, col_index=1)
    df_result = cut_dataframe_target(df_merged_1, 1, target_value)

    result_path = os.path.join(path, f"liabilitas_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.


Cannot set gray non-stroke color because /'P28' is an invalid float value
Cannot set gray non-stroke color because /'P30' is an invalid float value


Found exact match 'LIABILITAS' in Page 1, Table 1. Saved as 'LIABILITAS_page1_table1'.


Cannot set gray non-stroke color because /'P28' is an invalid float value
Cannot set gray non-stroke color because /'P30' is an invalid float value


Found exact match 'LIABILITAS' in Page 1, Table 1. Saved as 'LIABILITAS_page1_table1'.


Cannot set gray non-stroke color because /'P28' is an invalid float value
Cannot set gray non-stroke color because /'P30' is an invalid float value


Found exact match 'LIABILITAS' in Page 1, Table 1. Saved as 'LIABILITAS_page1_table1'.


### Seabank

In [70]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Seabank'

# PDF list to be processed
pdf_list = [
    "Seabankq42024.pdf",
    "Seabankq32024.pdf",
    "Seabankq22024.pdf",
    "Seabankq12024.pdf",
    "Seabankq22023.pdf",
    "Seabankq12023.pdf"
]

rasio_1_row_col_1_merge = [
    "Tagihan atas surat berharga yang dibeli dengan janji dijual kembali (reverse ",
    "7. Tagihan atas surat berharga yang dibeli dengan janji dijual kembali (reverse"
]

# Your search parameters
target_value = "LIABILITAS"       # The value to search for
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=200, #50
        row_tol=7, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    print("Processing PDF:", pdf_name)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_liabilitas = found_dfs[found_keys[0]] if found_keys else None
    print("Processing PDF:", pdf_name)
    if(len(found_keys) > 0) :
        print(f"Found {len(found_keys)} keys in {pdf_name}: {found_keys}")

    # Apply the function
    # df_merged_1 = merge_rows(df_liabilitas.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    # df_result = merge_rows(df_merged_1.copy(), rasio_1_row_col_1_merge, col_index=0)
    df_result = df_liabilitas.copy()

    result_path = os.path.join(path, f"liabilitas_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


Processing PDF: Seabankq42024.pdf
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Processing PDF: Seabankq42024.pdf
Found 1 keys in Seabankq42024.pdf: ['LIABILITAS_page2_table1']


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value
d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (36.919999999999995, 214.02398899999994, 578.9808444677496, 416.2054914695122)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)


Processing PDF: Seabankq32024.pdf
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Processing PDF: Seabankq32024.pdf
Found 1 keys in Seabankq32024.pdf: ['LIABILITAS_page2_table1']


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


...

Processing PDF: Seabankq12024.pdf
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Processing PDF: Seabankq12024.pdf
Found 1 keys in Seabankq12024.pdf: ['LIABILITAS_page2_table1']


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


Processing PDF: Seabankq22023.pdf
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Processing PDF: Seabankq22023.pdf
Found 1 keys in Seabankq22023.pdf: ['LIABILITAS_page2_table1']


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


Processing PDF: Seabankq12023.pdf
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Processing PDF: Seabankq12023.pdf
Found 1 keys in Seabankq12023.pdf: ['LIABILITAS_page2_table1']


In [71]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Seabank'

# PDF list to be processed
pdf_list = [
    "Seabankq42023.pdf",
    "Seabankq32023.pdf"
]

rasio_1_row_col_1_merge = [
    "Tagihan atas surat berharga yang dibeli dengan janji dijual kembali (reverse ",
    "7. Tagihan atas surat berharga yang dibeli dengan janji dijual kembali (reverse"
]

# Your search parameters
target_value = "LIABILITAS"       # The value to search for
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=50, #50
        row_tol=7, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    print("Processing PDF:", pdf_name)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_liabilitas = found_dfs[found_keys[0]] if found_keys else None
    print("Processing PDF:", pdf_name)
    if(len(found_keys) > 0) :
        print(f"Found {len(found_keys)} keys in {pdf_name}: {found_keys}")

    # Apply the function
    # df_merged_1 = merge_rows(df_liabilitas.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    # df_result = merge_rows(df_merged_1.copy(), rasio_1_row_col_1_merge, col_index=0)
    df_result = df_liabilitas.copy()

    result_path = os.path.join(path, f"liabilitas_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


Processing PDF: Seabankq42023.pdf
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Processing PDF: Seabankq42023.pdf
Found 1 keys in Seabankq42023.pdf: ['LIABILITAS_page2_table1']


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


Processing PDF: Seabankq32023.pdf
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Processing PDF: Seabankq32023.pdf
Found 1 keys in Seabankq32023.pdf: ['LIABILITAS_page2_table1']


### Allo Bank

In [82]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Allobank'

# PDF list to be processed
pdf_list = [
    "Allobankq42024.pdf",
    "Allobankq32024.pdf",
    "Allobankq22024.pdf",
    "Allobankq12024.pdf",
    "Allobankq42023.pdf",
    "Allobankq32023.pdf",
    "Allobankq22023.pdf",
    "Allobankq12023.pdf"
]

rasio_1_row_col_1_merge = [
    "Tagihan atas surat berharga yang dibeli dengan janji dijual kembali",
]

# Your search parameters
target_value = "LIABILITAS"       # The value to search for
row_range = (0, 10)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=200,
        row_tol=8, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_liabilitas = found_dfs[found_keys[0]] if found_keys else None

    # Apply the function
    # df_merged_1 = merge_rows(df_liabilitas.copy(), rasio_1_row_col_1_merge, col_index=1)
    # # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    # df_result = merge_rows(df_merged_1.copy(), rasio_1_row_col_1_merge, col_index=0)
    df_result = df_liabilitas.copy()

    result_path = os.path.join(path, f"liabilitas_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'LIABILITAS' in Page 2, Table 2. Saved as 'LIABILITAS_page2_table2'.


d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Found exact match 'LIABILITAS' in Page 2, Table 2. Saved as 'LIABILITAS_page2_table2'.


d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Found exact match 'LIABILITAS' in Page 2, Table 2. Saved as 'LIABILITAS_page2_table2'.


d:\anaconda\envs\camelot_env\lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-14 is image-based, camelot only works on text-based pages.
  warnings.warn(


Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Found exact match 'LIABILITAS' in Page 2, Table 2. Saved as 'LIABILITAS_page2_table2'.
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Found exact match 'LIABILITAS' in Page 2, Table 2. Saved as 'LIABILITAS_page2_table2'.
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Found exact match 'LIABILITAS' in Page 2, Table 2. Saved as 'LIABILITAS_page2_table2'.
Found exact match 'LIABILITAS' in Page 2, Table 1. Saved as 'LIABILITAS_page2_table1'.
Found exact match 'LIABILITAS' in Page 2, Table 2. Saved as 'LIABILITAS_page2_table2'.


### Bank Jago

In [16]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Bank Jago'

# PDF list to be processed
pdf_list = [
    "BankJagoq42024.pdf",
    "BankJagoq12024.pdf",
    "BankJagoq42023.pdf",
    "BankJagoq22023.pdf",
    "BankJagoq12023.pdf"
]

rasio_1_row_col_1_merge = [
    "Surat berharga yang dijual dengan janji dibeli",
]

# Your search parameters
# target_value = "RASIO (%)"       # The value to search for
target_value = "LIABILITAS"
row_range = (0, 40)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=50,
        row_tol=5, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    print("Processing PDF:", pdf_name)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_liabilitas = found_dfs[found_keys[0]] if found_keys else None
    if(len(found_keys) > 0) :
        print(f"Found {len(found_keys)} keys in {pdf_name}: {found_keys}")

    # Apply the function
    # df_merged_1 = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    df_result = cut_dataframe_target(df_liabilitas, 1, target_value)

    result_path = os.path.join(path, f"liabilitas_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Processing PDF: BankJagoq42024.pdf
Found exact match 'LIABILITAS' in Page 1, Table 1. Saved as 'LIABILITAS_page1_table1'.
Found 1 keys in BankJagoq42024.pdf: ['LIABILITAS_page1_table1']
Processing PDF: BankJagoq12024.pdf
Found exact match 'LIABILITAS' in Page 1, Table 1. Saved as 'LIABILITAS_page1_table1'.
Found 1 keys in BankJagoq12024.pdf: ['LIABILITAS_page1_table1']
Processing PDF: BankJagoq42023.pdf
Found exact match 'LIABILITAS' in Page 1, Table 1. Saved as 'LIABILITAS_page1_table1'.
Found 1 keys in BankJagoq42023.pdf: ['LIABILITAS_page1_table1']
Processing PDF: BankJagoq22023.pdf
Found exact match 'LIABILITAS' in Page 1, Table 1. Saved as 'LIABILITAS_page1_table1'.
Found 1 keys in BankJagoq22023.pdf: ['LIABILITAS_page1_table1']
Processing PDF: BankJagoq12023.pdf
Found exact match 'LIABILITAS' in Page 1, Table 1. Saved as 'LIABILITAS_page1_table1'.
Found 1 keys in BankJagoq12023.pdf: ['LIABILITAS_page1_table1']


In [19]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\Bank Jago'

# PDF list to be processed
pdf_list = [
    "BankJagoq32023.pdf",
    "BankJagoq22024.pdf",
    "BankJagoq32024.pdf",
]

rasio_1_row_col_1_merge = [
    "7.  Tagihan atas surat berharga yang dibeli dengan janji",
    "13. Cadangan kerugian penurunan nilai aset"
]

# Your search parameters
# target_value = "RASIO (%)"       # The value to search for
target_value = "LIABILITAS"
row_range = (0, 40)         # Check only rows from index 0 to 10 (inclusive)
col_range = (0, 2)          # Check only columns from index 0 to 2 (inclusive)

for pdf_name in pdf_list:
    pdf_path = os.path.join(path, pdf_name)
    # Extract tables from all pages using 'stream' mode
    tables = camelot.read_pdf(
        pdf_path, 
        flavor="stream", 
        pages="all", 
        edge_tol=20,
        row_tol=5, 
        strip_text='\n',
    )
    page_dfs = extract_pdf(tables)
    print("Processing PDF:", pdf_name)
    found_dfs, found_keys, found_page_nums = search_page(page_dfs, target_value, row_range, col_range)
    df_liabilitas = found_dfs[found_keys[0]] if found_keys else None
    if(len(found_keys) > 0) :
        print(f"Found {len(found_keys)} keys in {pdf_name}: {found_keys}")

    # Apply the function
    # df_merged_1 = merge_rows(df_rasio.copy(), rasio_1_row_col_1_merge, col_index=1)
    # df_merged_2 = merge_2_rows(df_merged_1.copy(), rasio_2_row_col_1_merge, col_index=1)
    df_result = cut_dataframe_target(df_liabilitas, 1, target_value)

    result_path = os.path.join(path, f"liabilitas_merged_{pdf_name.replace('.pdf', '.xlsx')}")
    df_result.to_excel(
        result_path, 
        index=False, 
        header=False
    )

Processing PDF: BankJagoq32023.pdf
Found exact match 'LIABILITAS' in Page 1, Table 1. Saved as 'LIABILITAS_page1_table1'.
Found 1 keys in BankJagoq32023.pdf: ['LIABILITAS_page1_table1']
Processing PDF: BankJagoq22024.pdf
Found exact match 'LIABILITAS' in Page 1, Table 1. Saved as 'LIABILITAS_page1_table1'.
Found 1 keys in BankJagoq22024.pdf: ['LIABILITAS_page1_table1']
Processing PDF: BankJagoq32024.pdf
Found exact match 'LIABILITAS' in Page 1, Table 1. Saved as 'LIABILITAS_page1_table1'.
Found 1 keys in BankJagoq32024.pdf: ['LIABILITAS_page1_table1']
